In [1]:
from datasets import load_dataset, load_from_disk
import os
import sys
print(os.getcwd())
print(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.getcwd()))
# print(sys.path)
from src.utils import translate_to_phonetic
from src.train_tokenizer import load_tokenizer
from src.bert_wikitext_train import preprocess_dataset

/home/toure215/BERT_phonetic/test
/home/toure215/BERT_phonetic


In [14]:
bookcorpus = load_from_disk("/home/toure215/BERT_phonetic/DATASETS/phonetic_bookcorpus")

Loading dataset from disk:   0%|          | 0/47 [00:00<?, ?it/s]

In [15]:
bookcorpus

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 113445651
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 224959
    })
})

In [16]:
print(bookcorpus["train"][0])

{'text': 'wVt '}


In [19]:
text_lengths = bookcorpus['train'].map(lambda x: {"text_length": len(x["text"][0].split(" "))}, num_proc=16, remove_columns=["text"])
total_length = sum(text_lengths["text_length"])
mean = total_length / len(bookcorpus["train"])
mean

1.0566846938892351

In [24]:
print(text_lengths[:10])
bookcorpus["train"][6]

{'text_length': [1, 1, 1, 1, 1, 1, 2, 1, 1, 1]}


{'text': ' wi S{l bi Ol r\\ajt wEr\\ wi Ar\\  '}

In [7]:
tokenizer = load_tokenizer("/home/toure215/BERT_phonetic/tokenizers/tokenizer_phonetic_BPE")
tokenized_dataset = preprocess_dataset(bookcorpus, tokenizer, 128)

Loading tokenizer from tokenizer_phonetic_BPE


In [8]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 113445651
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 224959
    })
})

In [9]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

datacollator = DataCollatorWithPadding(tokenizer)
dataloader = DataLoader(tokenized_dataset["train"], batch_size=8, num_workers=16, collate_fn=datacollator)
for step, batch in enumerate(dataloader):
    print(batch["input_ids"].shape)
    if step > 10:
        break

torch.Size([8, 128])
torch.Size([8, 128])
torch.Size([8, 128])
torch.Size([8, 40])
torch.Size([8, 128])
torch.Size([8, 128])
torch.Size([8, 33])
torch.Size([8, 128])
torch.Size([8, 128])
torch.Size([8, 128])
torch.Size([8, 128])
torch.Size([8, 128])


In [10]:
bookcorpus = load_from_disk("/home/toure215/BERT_phonetic/DATASETS/bookcorpus")
bookcorpus

Loading dataset from disk:   0%|          | 0/51 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 80301972
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 160926
    })
})

In [11]:
text_lengths = bookcorpus['train'].map(lambda x: {"text_length": len(x["text"][0].split(" "))}, num_proc=16, remove_columns=["text"])
total_length = sum(text_lengths["text_length"])
mean = total_length / len(bookcorpus["train"])
mean

19.792904737133977

In [12]:
print(text_lengths[:100])
print(max(text_lengths["text_length"]))

{'text_length': [18, 7, 10, 6, 5, 18, 15, 7, 39, 27, 33, 7, 7, 9, 9, 10, 4, 21, 6, 6, 6, 14, 9, 6, 15, 12, 9, 7, 12, 9, 16, 9, 10, 12, 27, 21, 5, 91, 8, 10, 3, 3, 11, 5, 17, 3, 16, 12, 6, 35, 49, 10, 34, 4, 18, 5, 22, 6, 19, 13, 36, 8, 8, 36, 5, 23, 6, 19, 17, 3, 23, 22, 10, 29, 20, 6, 9, 14, 100, 28, 31, 6, 8, 23, 12, 6, 12, 29, 3, 8, 10, 2, 100, 100, 5, 4, 100, 18, 6, 20]}
100
